In [1]:
import importlib

import numpy as np
import pandas as pd
import numpy
import lab_graph

importlib.reload(lab_graph)

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

df = pd.read_csv("data/all-colors-unique.csv")

#Create a 2D array of Lab values
lab_list = list(df["L*a*b Value"])
lab_list = [x.split(", ") for x in lab_list]
lab_points = [[float(x) for x in row] for row in lab_list]

#Use greedy approach
result = lab_graph.greedy_mdp(lab_points, 16)

#Create new dataframe with just the resultant indices
result_df = df.loc[result]

#lab_graph.generate_lab_3d_all_colors(result_df, "Euclidean", "N/a")

In [2]:
from scipy.spatial.distance import pdist, squareform
import colormath.color_objects as co
import colormath.color_diff as cdiff

euclidean_distances = squareform(pdist(lab_points))
#Retrieves the indices furthest points
i, j = np.unravel_index(np.argmax(euclidean_distances), euclidean_distances.shape)

print("___Euclidean Distance___")
print(df.iloc[i])
print(df.iloc[j])
# Use lab points

def lab_point_distance(color1, color2, delta_e="CIE2000"):
    """
    Compute the difference between two colors using a Delta E formula.

    :param color1: Lab color number 1
    :param color2: Lab color number 2
    :param delta_e: The comparison formula, CIE2000 (symmetric) is default
    :return: Returns the distance between two colors using the CIE2000 or CMC delta E formula
    """
    lab_1 = co.LabColor(color1[0], color1[1], color1[2])
    lab_2 = co.LabColor(color2[0], color2[1], color2[2])

    if delta_e == "CIE2000":
        return cdiff.delta_e_cie2000(lab_1, lab_2, Kl=2, Kc=1, Kh=1)
    if delta_e == "CMC":
        return cdiff.delta_e_cmc(lab_1, lab_2, pl=2, pc=1)

distances_delta_e = squareform(pdist(lab_points, metric=lab_point_distance))
i,j = np.unravel_index(np.argmax(distances_delta_e), distances_delta_e.shape)

print("\n___Delta_E_Distance___")
print(df.iloc[i])
print(df.iloc[j])


___Euclidean Distance___
Color Name                      Poppy
PMS Color Code                  2347C
L*a*b Value       49.45, 75.12, 67.21
Hex                            E10600
Name: 36, dtype: object
Color Name                         Teal
PMS Color Code                    3262C
L*a*b Value       66.44, -59.33, -12.05
Hex                              00BFB2
Name: 46, dtype: object

___Delta_E_Distance___
Color Name                      Berry
PMS Color Code                  233CP
L*a*b Value       45.38, 71.84, -7.30
Hex                            C6057B
Name: 6, dtype: object
Color Name               Heather Kelly
PMS Color Code                   340 C
L*a*b Value       51.80, -65.36, 14.55
Hex                             00965E
Name: 72, dtype: object


1. List of LAB points
2. Generate a distance matrix using Euclidean/Delta E distance
3. Select a pair of points (starting point for greedy algorithm)
4. Run greedy algorithm on distance matrix
5. Create a resultant dataframe
6. Calculate a score based off the result

In [3]:
def greedy_search(points, distance_matrix, number_of_teams, start_a, start_b):
    """
    Returns the list of indices corresponding to the colors in the dataframe found
    through the greedy search
    :param points: Set of lab color points the greedy search is being performed in
    :param distance_matrix: Matrix of distances between points
    :param number_of_teams: Total number of teams optimizing for
    :param start_a: Index of the color in the dataframe, distance matrix, and point set
    :param start_b: index of the color in the dataframe, distance matrix, and point set
    :return:
    """
    n = len(points)
    k = number_of_teams #Total number of teams
    selected = set()
    selected.add(start_a)
    selected.add(start_b)

    while len(selected) < k:
        max_min_dist = -1
        best_point = -1

        #For i in the set of all colors, index of a specific color
        for i in range(n):
            # If the index has not been chosen
            if i not in selected:
                #Find the point with the greatest min distance from those selected, provides an even distribution
                #This does not find the furthest point from the current point, it's better than that
                min_dist = min(distance_matrix[i][j] for j in selected)
                if min_dist > max_min_dist:
                    max_min_dist = min_dist
                    best_point = i

        selected.add(best_point)

    return list(selected)

def score(distance_matrix, chosen):
    """
    Scores the resultant set of points based off the average distance between all chosen points
    :param distance_matrix: Matrix representing the distances between colors
    :param chosen: Chosen points (from the set of colors)
    :return: The average distance between all the provided (chosen) colors
    """
    subset_distances = distance_matrix[np.ix_(chosen, chosen)]
    upper_triangle = np.triu(subset_distances, k=1)
    avg_distance = upper_triangle.sum() / (upper_triangle != 0).sum()

    return avg_distance

point_a, point_b = np.unravel_index(np.argmax(distances_delta_e), distances_delta_e.shape)
selected = greedy_search(lab_points, distances_delta_e, 16, point_a, point_b)

delta_e_score = score(distances_delta_e, selected)
euclidean_score = score(euclidean_distances, result)

print("Delta E: " + str(delta_e_score))
print("Euclidean: " + str(euclidean_score))

Delta E: 38.72694104952147
Euclidean: 78.72508932265154


Delta E: 38.77662607204613
Euclidean: 78.72508932265154

In [4]:
#Retrieve the hundred furthest points
#check if distances is referenced anywhere in here
def n_furthest_points(number_of_points, distance_matrix):
    """
    Computes the n furthest points
    :param number_of_points: The nth point you want computed
    :param distance_matrix: The distance matrix representing the distances between all points in the set
    :return: The most distant points in the matrix, from 0 to n
    """
    flat_indices = np.argpartition(distance_matrix.flatten(), -number_of_points)[-number_of_points:]
    # Convert to 2D indices
    i_indices, j_indices = np.unravel_index(flat_indices, distance_matrix.shape)
    # Get corresponding distances
    top_distances = distance_matrix[i_indices, j_indices]
    # Sort by distance (optional)
    sort_idx = np.argsort(top_distances)[::-1]
    i_indices = i_indices[sort_idx]
    j_indices = j_indices[sort_idx]
    top_distances = top_distances[sort_idx]

    return i_indices, j_indices, top_distances

#i_points, j_points, top_dist = n_furthest_points(100, euclidean_distances)

In [5]:
top_euclidean_sets = []
scores = []

df = pd.read_csv("data/all-colors-unique.csv")
euclidean_distances = squareform(pdist(lab_points))

lab_list = list(df["L*a*b Value"])
lab_list = [x.split(", ") for x in lab_list]
lab_points = [[float(x) for x in row] for row in lab_list]

#103 Choose 2 is 5253, which we can compute, however every item is represented twice, so it's actually 10,506
i_points, j_points, top_dist = n_furthest_points(10506, euclidean_distances)

total_teams = 16

for run in range(10506):
    search_result = greedy_search(lab_points, euclidean_distances, total_teams, i_points[run], j_points[run])

    search_score = score(euclidean_distances, search_result)

    top_euclidean_sets += [search_result]

    scores += [search_score]


# Greedy Search Results

In [6]:
scores.sort()
print("Lowest: " + str(scores[0]))
print("Highest: " + str(scores [-1]))

scores_set = set(scores)
len(scores_set)

Lowest: 72.54711616260738
Highest: 81.92102523616252


4440

In [7]:
# how do the above scores compare to randomly selecting points
import random
random_tuples = set()

while len(random_tuples) < 10506:
    random_tuples.add(tuple(random.sample(range(0, 102), 16)))


In [8]:
random_scores = []
for random_tuple in random_tuples:
    random_score = score(euclidean_distances, list(random_tuple))
    random_scores += [random_score]


# Random Search Results

In [9]:
random_scores.sort()
len(random_scores)
print("Lowest: " + str(random_scores[0]))
print("Highest: " + str(random_scores[-1]))

Lowest: 33.096185624646225
Highest: 75.32804624466026


In [94]:
# Let's come up with a better scoring metric
import colormath.color_conversions as color_convert
from colormath.color_objects import sRGBColor

all_colors = []
exclusion_array = []
y_step = 4
for y in range(0,101, y_step):
    for x in range(-127, 127, 24):
        for z in range(-127, 127, 24):

            #If we have excluded this column, continue
            current_xz = [x, z]
            if current_xz in exclusion_array:
                continue

            color_name = " "
            pms_color_code = " "
            lab_value = str(y) + ", " + str(x) + ", " + str(z)

            lab_color = co.LabColor(y, x, z)

            if y < 100:
                lab_color_above = co.LabColor(y + y_step, x, z)
                lab_diff = cdiff.delta_e_cie2000(lab_color, lab_color_above)
                if lab_diff > 2.82: #maybe change to 2.82, play with this
                    exclusion_array += [[x, z]]
                    continue

            rgb_color = color_convert.convert_color(lab_color, sRGBColor)
            hex_color = sRGBColor.get_rgb_hex(rgb_color)

            all_colors += [[color_name, pms_color_code, lab_value, hex_color[1:7].upper(), rgb_color, lab_color]]
exclusion_array

[[-127, -127],
 [-127, -103],
 [-127, -79],
 [-127, -55],
 [-127, -31],
 [-127, -7],
 [-127, 17],
 [-127, 41],
 [-127, 65],
 [-127, 89],
 [-127, 113],
 [-103, -127],
 [-103, -103],
 [-103, -79],
 [-103, -55],
 [-103, -31],
 [-103, -7],
 [-103, 17],
 [-103, 41],
 [-103, 65],
 [-103, 89],
 [-103, 113],
 [-79, -127],
 [-79, -103],
 [-79, -79],
 [-79, -55],
 [-79, -31],
 [-79, -7],
 [-79, 17],
 [-79, 41],
 [-79, 65],
 [-79, 89],
 [-79, 113],
 [-55, -127],
 [-55, -103],
 [-55, -79],
 [-55, -55],
 [-55, -31],
 [-55, -7],
 [-55, 17],
 [-55, 41],
 [-55, 65],
 [-55, 89],
 [-55, 113],
 [-31, -127],
 [-31, -103],
 [-31, -79],
 [-31, -55],
 [-31, -31],
 [-31, -7],
 [-31, 17],
 [-31, 41],
 [-31, 65],
 [-31, 89],
 [-31, 113],
 [-7, -127],
 [-7, -103],
 [-7, -79],
 [-7, -55],
 [-7, -31],
 [-7, -7],
 [-7, 17],
 [-7, 41],
 [-7, 65],
 [-7, 89],
 [-7, 113],
 [17, -127],
 [17, -103],
 [17, -79],
 [17, -55],
 [17, -31],
 [17, -7],
 [17, 17],
 [17, 41],
 [17, 65],
 [17, 89],
 [17, 113],
 [41, -127],
 [41, -

In [95]:
importlib.reload(lab_graph)

full_space = pd.DataFrame(all_colors, columns=["Color Name", "PMS Color Code", "L*a*b Value", "Hex", "RGB", "LAB"])


,Color Name,PMS Color Code,L*a*b Value,Hex,RGB,LAB
600,,,"16, 113, 17",9F001C,sRGBColor (rgb_r:0.6224 rgb_g:0.0000 rgb_b:0.1...,LabColor (lab_l:16.0000 lab_a:113.0000 lab_b:1...
601,,,"16, 113, 41",A00000,sRGBColor (rgb_r:0.6274 rgb_g:0.0000 rgb_b:0.0...,LabColor (lab_l:16.0000 lab_a:113.0000 lab_b:4...
602,,,"16, 113, 65",A10000,sRGBColor (rgb_r:0.6325 rgb_g:0.0000 rgb_b:0.0...,LabColor (lab_l:16.0000 lab_a:113.0000 lab_b:6...
603,,,"16, 113, 89",A30000,sRGBColor (rgb_r:0.6374 rgb_g:0.0000 rgb_b:0.0...,LabColor (lab_l:16.0000 lab_a:113.0000 lab_b:8...
604,,,"16, 113, 113",A40000,sRGBColor (rgb_r:0.6424 rgb_g:0.0000 rgb_b:0.0...,LabColor (lab_l:16.0000 lab_a:113.0000 lab_b:1...


In [93]:
lab_graph.generate_lab_3d_all_colors(full_space, "test", "n/a")

In [70]:
full_space.head()

,Color Name,PMS Color Code,L*a*b Value,Hex,RGB,LAB
0,,,"0, -127, -127",003BC1,sRGBColor (rgb_r:0.0000 rgb_g:0.2308 rgb_b:0.7...,LabColor (lab_l:0.0000 lab_a:-127.0000 lab_b:-...
1,,,"0, -127, -103",003799,sRGBColor (rgb_r:0.0000 rgb_g:0.2165 rgb_b:0.6...,LabColor (lab_l:0.0000 lab_a:-127.0000 lab_b:-...
2,,,"0, -127, -79",003474,sRGBColor (rgb_r:0.0000 rgb_g:0.2059 rgb_b:0.4...,LabColor (lab_l:0.0000 lab_a:-127.0000 lab_b:-...
3,,,"0, -127, -55",003350,sRGBColor (rgb_r:0.0000 rgb_g:0.1988 rgb_b:0.3...,LabColor (lab_l:0.0000 lab_a:-127.0000 lab_b:-...
4,,,"0, -127, -31",00322E,sRGBColor (rgb_r:0.0000 rgb_g:0.1947 rgb_b:0.1...,LabColor (lab_l:0.0000 lab_a:-127.0000 lab_b:-...


In [50]:
#004DEE vs 0055FA vs 005D10

#distances_delta_e = squareform(pdist(lab_points, metric=lab_point_distance))

#Convert rgb colors to a list of vectors

from scipy.spatial import distance

color_list = []
for color in full_space["RGB"]:
    r = color.rgb_r
    g = color.rgb_g
    b = color.rgb_b

    color_list += [[r,g,b]]

rgb_distances = squareform(pdist(color_list))

removal_array = []
#TODO I need to define these in variables or I'm going to keep messing this up
for y in range(0,101, 4):
    for x in range(-127, 127, 24):
        for z in range(-127, 127, 24):
            # Get the current item
            # Get the item above (if possible)
            lab_value = str(y) + ", " + str(x) + ", " + str(z)
            current = full_space.loc[full_space["L*a*b Value"] == lab_value]
            current_rgb = [current.iloc[0].RGB.rgb_r, current.iloc[0].RGB.rgb_g, current.iloc[0].RGB.rgb_b]

            if y + 4 < 100:
                lab_value_above = str(y + 4) + ", " + str(x) + ", " + str(z)
                above = full_space.loc[full_space["L*a*b Value"] == lab_value_above]
                above_rgb = [above.iloc[0].RGB.rgb_r, above.iloc[0].RGB.rgb_g, above.iloc[0].RGB.rgb_b]
            else:
                continue

            color_distance = distance.euclidean(current_rgb, above_rgb)

            if color_distance >= 1:
                print("The distance is greater than 1")
                removal_array += [[y, x, z]]

len(removal_array)
#rgb_distances




0

In [78]:
current = full_space.loc[full_space["L*a*b Value"] == "20, 41, -127"]
above = full_space.loc[full_space["L*a*b Value"] == "24, 41, -127"]

#print(current["Hex"], "\n", above["Hex"])
c_rgb = [current.iloc[0].RGB.rgb_r, current.iloc[0].RGB.rgb_g, current.iloc[0].RGB.rgb_b]
a_rgb = [above.iloc[0].RGB.rgb_r, above.iloc[0].RGB.rgb_g, above.iloc[0].RGB.rgb_b]

d = distance.euclidean(c_rgb, a_rgb)

c_lab = current.iloc[0].LAB
a_lab = above.iloc[0].LAB

d_lab = cdiff.delta_e_cie2000(c_lab, a_lab)

print(d_lab)
#for hex colors, this will not work
# 0.055044 for similar
# 0.057716 for close

#for lab color difference CIE 2000
# far is 2.827
# close is 2.711
# close is 2.604
#ar is 2.82736

2.8273682018368325


In [51]:

# bottom = 0x004DEE #(00, 77, 238)
# middle = 0x0055FA # (00, 85, 250)
# top = 0x005D10 #(0,93, 16)

bottom = (00, 77, 238)
middle = (00, 85, 250)
top = (0,93, 16)

close = distance.euclidean(middle, bottom)
far = distance.euclidean(top, middle)

print(close)
print(far)

# Use vectors to compute the RGB difference, pure hex difference is no good

# For optimal space -> Plot the volume, and then collapse it onto the current set of points?
# Also try and create a super low poly version of the full space and use that as the "target" difference

14.422205101855956
234.13671220037236
